# **GIS × Python Tutorial 8.1 ~スタイル定義済みKMLの作成~**

<br>

## **はじめに**
この記事は「GIS × Python Tutorial」の関連記事です。

今回の`8.x`では `GeoPandas.GeoDataFrame`のクラスを拡張して、スタイルを定義したKMLを出力する新たなクラスを作成してみたいと思います。GeoDataFrame ではすでに `GeoDataFrame.to_file` で KML を出力する事が可能ですが、せっかく自由度が高い KML を使用するのであれば、スタイルを定義して KML を出力したいと思いこの記事を書いています。

チュートリアルは以下の順番で進んで行きます。

 - [8.1] fastkml での KML 作成

 - [8.2] GeoDataFrame の継承と独自メソッドの追加

本来であれば、様々な Geometry に対応させるべきですが、今回は面倒なので Polygon | MultiPolygon に絞ってデータを作成していきます。

In [170]:
from pprint import pprint

import geopandas as gpd
import fastkml
import pyproj
import pygeoif
import shapely

In [171]:
kml = fastkml.kml.KML()

# Add document.
doc = fastkml.kml.Document(id='learngis', name='Learn GIS', description='Learn GIS')
kml.append(doc)
print(kml.to_string(prettyprint=True))

<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document id="learngis">
    <name>Learn GIS</name>
    <description>Learn GIS</description>
  </Document>
</kml>



### Styling

In [172]:
icon_style = fastkml.styles.IconStyle(
    color='ff0000ff',
    icon_href='http://maps.google.com/mapfiles/kml/paddle/red-stars.png'
)
print(icon_style.to_string(prettyprint=True))

<kml:IconStyle xmlns:kml="http://www.opengis.net/kml/2.2">
  <kml:color>ff0000ff</kml:color>
  <kml:Icon>
    <kml:href>http://maps.google.com/mapfiles/kml/paddle/red-stars.png</kml:href>
  </kml:Icon>
</kml:IconStyle>



In [173]:
sparsely_style = fastkml.styles.Style(id='sparselyPop', styles=[icon_style])

print(sparsely_style.to_string(prettyprint=True))

<kml:Style xmlns:kml="http://www.opengis.net/kml/2.2" id="sparselyPop">
  <kml:IconStyle>
    <kml:color>ff0000ff</kml:color>
    <kml:Icon>
      <kml:href>http://maps.google.com/mapfiles/kml/paddle/red-stars.png</kml:href>
    </kml:Icon>
  </kml:IconStyle>
</kml:Style>



### Create folder 

In [174]:
# Add folder.
folder = fastkml.kml.Folder(id='prefecture', name='Prefecture', description='Prefectures')
folder.append(sparsely_style)

print(folder.to_string(prettyprint=True))

<kml:Folder xmlns:kml="http://www.opengis.net/kml/2.2" id="prefecture">
  <kml:name>Prefecture</kml:name>
  <kml:description>Prefectures</kml:description>
  <kml:Style id="sparselyPop">
    <kml:IconStyle>
      <kml:color>ff0000ff</kml:color>
      <kml:Icon>
        <kml:href>http://maps.google.com/mapfiles/kml/paddle/red-stars.png</kml:href>
      </kml:Icon>
    </kml:IconStyle>
  </kml:Style>
</kml:Folder>



### Extended data

In [175]:
data = [
    fastkml.data.Data(name='prefecture', value='Aomori'),
    fastkml.data.Data(name='population', value='118.8'),
]
extended_data = fastkml.data.ExtendedData(elements=data)

print(extended_data.to_string(prettyprint=True))

<kml:ExtendedData xmlns:kml="http://www.opengis.net/kml/2.2">
  <kml:Data name="prefecture">
    <kml:value>Aomori</kml:value>
  </kml:Data>
  <kml:Data name="population">
    <kml:value>118.8</kml:value>
  </kml:Data>
</kml:ExtendedData>



### Placemark

In [176]:
# shapely.geometry -> pygeoif.geometry
shapely_pnt = shapely.geometry.Point(140.000, 40.000)
pygeoif_pnt = pygeoif.shape(shapely_pnt)
print(f"type: {type(pygeoif_pnt)}, geometry: {pygeoif_pnt}")

type: <class 'pygeoif.geometry.Point'>, geometry: POINT (140.0 40.0)


In [177]:
placemark = fastkml.features.Placemark(
    id='1',
    name='Aomori',
    geometry=pygeoif_pnt,
    extended_data=extended_data,
    style_url=fastkml.styles.StyleUrl(url=f"#{sparsely_style.id}")
)
print(placemark.to_string(prettyprint=True))

<kml:Placemark xmlns:kml="http://www.opengis.net/kml/2.2" id="1">
  <kml:name>Aomori</kml:name>
  <kml:styleUrl>#sparselyPop</kml:styleUrl>
  <kml:ExtendedData>
    <kml:Data name="prefecture">
      <kml:value>Aomori</kml:value>
    </kml:Data>
    <kml:Data name="population">
      <kml:value>118.8</kml:value>
    </kml:Data>
  </kml:ExtendedData>
  <kml:Point>
    <kml:coordinates>140.0,40.0</kml:coordinates>
  </kml:Point>
</kml:Placemark>



In [178]:
folder.append(placemark)
doc.append(folder)
print(kml.to_string(prettyprint=True).replace('  ', '    '))

<kml xmlns="http://www.opengis.net/kml/2.2">
    <Document id="learngis">
        <name>Learn GIS</name>
        <description>Learn GIS</description>
        <Folder id="prefecture">
            <name>Prefecture</name>
            <description>Prefectures</description>
            <Style id="sparselyPop">
                <IconStyle>
                    <color>ff0000ff</color>
                    <Icon>
                        <href>http://maps.google.com/mapfiles/kml/paddle/red-stars.png</href>
                    </Icon>
                </IconStyle>
            </Style>
            <Placemark id="1">
                <name>Aomori</name>
                <styleUrl>#sparselyPop</styleUrl>
                <ExtendedData>
                    <Data name="prefecture">
                        <value>Aomori</value>
                    </Data>
                    <Data name="population">
                        <value>118.8</value>
                    </Data>
                </ExtendedData>
     